In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def evaluate_rules(data, tuned_rules, output):
    predictions_all = []
    for i, rule in tuned_rules.iterrows():
        pred_rule = []
        tptot = 0
        tntot = 0
        fntot = 0
        fptot = 0
        tuned_antecedent = rule['Rule'].strip()
        print(f"rule: {tuned_antecedent}")
        for _, row in data.iterrows():
            y_row = row[output]
            tp = 0
            tn = 0
            fn = 0
            fp = 0
            # check if the point row satifies rule 
            if evaluate_rule_conditions(row, tuned_antecedent):
                # rule is satisfied
                satisfied = True
                pred_rule.append(0)
            else:
                satisfied = False
                pred_rule.append(1)
    
            
            if satisfied and y_row == 0:
                tp=1
            elif satisfied and y_row == 1:
                fp=1
            elif not satisfied and y_row == 1:
                tn=1
            
            elif not satisfied and y_row == 0:
                fn=1

            predictions_all.append(pred_rule)
            tptot+=tp
            tntot+=tn
            fptot+=fp
            fntot+=fn

        if (tptot+fptot) == 0:
            print("no points satisfy the rule\n")
            continue
        else:
            
            print(f"tp = {tptot}, tn = {tntot}, fp = {fptot}, fn = {fntot}")
            prec = tptot / (tptot+fptot)
            cov = tptot/(tptot+fntot)
            acc = (tptot+tntot)/(tptot+tntot+fptot+fntot)
            f1score = (2*tptot)/(2*tptot+fptot+fntot)
            error = fptot/(fptot+tntot)
            print(f"precision: {prec}, covering: {cov}, error: {error}, accuracy: {acc}, f1: {f1score}\n")              

    return predictions_all


def evaluate_rule_conditions(row, condition_part):

    # Checks if any of the conditions in the rule are satisfied
    if all(check_condition(row, part) for part in condition_part.split(" and ")):
        return True  # Return True if all conditions in the rule are satisfied
    
    return False  # Return False if any of the conditions in the rule is not satisfied


def check_condition(row, condition_part):
    # Check if a single condition part is satisfied
    
    #parts = [part.strip('()') for part in condition_part.split()]
    parts = re.split(r'\s*(==|<=|>=|<|>|!=)\s*', condition_part.strip('()'))
    #print("parts: ", parts)
    if len(parts) == 3:
        column, op, value = parts
        return eval(f"{row[column]} {op} {value}")
    # handle the case of a 2-thresholds conditions of the kind: a < Column <= b
    elif len(parts) == 5:
        val1,op1,column,op2,val2 = parts
        # Use the original condition from the rule
        return eval(f"{val1} {op1} {row[column]} {op2} {val2}")        
    else:
        # Handle cases where the condition is not in the expected format
        pass


# LLM performance evaluation rule by rule

In [3]:
# Loading rules actual rules
rules = pd.read_csv('/Users/saranarteni/OneDrive - CNR/REXASI-PRO/T3.1/navground_code/simulation2/confiderai/supsinavground_rules.csv', header=None, names=["Rule", "Covering", "Error"])

# adjust columns values
for i in range(len(rules)+1):   
    rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("AND","and"))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("{",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("}",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("output in ","output = "))
rules["Covering"] = rules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
rules["Error"] = rules["Error"].apply(lambda x: x.replace("ERROR: ",""))
rules['Output'] = rules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("IF ",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))
# load test data
data = pd.read_csv("simulation2/test.csv")

rules_noncoll = rules[rules["Output"]==0]
preds_LLM = evaluate_rules(data, rules_noncoll, "output")

rule: SafetyMargin > 0.071752 and Tau <= 0.798864
tp = 219, tn = 431, fp = 18, fn = 332
precision: 0.9240506329113924, covering: 0.397459165154265, error: 0.0400890868596882, accuracy: 0.65, f1: 0.5558375634517766

rule: SafetyMargin > 0.019271 and 0.096355 < Tau <= 0.359004
tp = 216, tn = 429, fp = 20, fn = 335
precision: 0.9152542372881356, covering: 0.39201451905626133, error: 0.044543429844097995, accuracy: 0.645, f1: 0.5489199491740788

rule: SafetyMargin > 0.070627 and Eta > 0.372908
tp = 192, tn = 441, fp = 8, fn = 359
precision: 0.96, covering: 0.3484573502722323, error: 0.017817371937639197, accuracy: 0.633, f1: 0.511318242343542

rule: 0.041272 < SafetyMargin <= 0.071004 and Tau <= 0.619041
tp = 142, tn = 434, fp = 15, fn = 409
precision: 0.9044585987261147, covering: 0.2577132486388385, error: 0.0334075723830735, accuracy: 0.576, f1: 0.4011299435028249

rule: SafetyMargin > 0.010931 and Eta > 0.748042 and Tau <= 0.682408
tp = 145, tn = 430, fp = 19, fn = 406
precision: 0.884

# Skope Rules performance

In [4]:
# Loading rules actual rules
skrules = pd.read_csv('/Users/saranarteni/OneDrive - CNR/REXASI-PRO/T3.1/navground_code/simulation2/skope/skope_rules.csv', header=None, names=["Rule", "Covering", "Error"])

# adjust columns values
for i in range(len(skrules)+1):   
    skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("AND","and"))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("{",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("}",""))
#skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("output in ","output = "))
skrules["Covering"] = skrules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
skrules["Error"] = skrules["Error"].apply(lambda x: x.replace("ERROR: ",""))
skrules['Output'] = skrules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("IF ",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))


skrules_noncoll = skrules[skrules["Output"]==0]
preds_skope = evaluate_rules(data, skrules_noncoll, "output")

rule: SafetyMargin > 0.03679436631500721 and Eta > 0.25215454399585724 and Tau <= 0.6308729648590088
tp = 284, tn = 441, fp = 8, fn = 267
precision: 0.9726027397260274, covering: 0.515426497277677, error: 0.017817371937639197, accuracy: 0.725, f1: 0.6737841043890866

rule: SafetyMargin > 0.057177115231752396 and Tau > 0.5994541049003601
tp = 116, tn = 389, fp = 60, fn = 435
precision: 0.6590909090909091, covering: 0.21052631578947367, error: 0.133630289532294, accuracy: 0.505, f1: 0.31911966987620355

rule: SafetyMargin > 0.03678788058459759 and Eta <= 0.3592268079519272 and Tau <= 0.6307087540626526
tp = 109, tn = 422, fp = 27, fn = 442
precision: 0.8014705882352942, covering: 0.19782214156079855, error: 0.060133630289532294, accuracy: 0.531, f1: 0.3173216885007278

rule: SafetyMargin <= 0.04115717485547066 and SafetyMargin > 0.0211033895611763 and Tau <= 0.34770750999450684
tp = 64, tn = 432, fp = 17, fn = 487
precision: 0.7901234567901234, covering: 0.1161524500907441, error: 0.0378